# This Notebook is going to be used for the Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Extracting and Processing HTML data, then writing it into a Data Frame

In [3]:
#import libraries needed for extracting and processing html data
import urllib.request, urllib.parse,urllib.error
from bs4 import BeautifulSoup

Now the data is pulled from from the wikipedia page in the shared URL

In [4]:
# pulling the html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'html.parser')

Now the data from the link is parsed to variables we have setup

In [5]:
#Retrieving data from the html
table = soup.find('table',attrs = { 'class' : 'wikitable sortable jquery-tablesorter'})
table_body = soup.find('tbody')
rows = table_body.find_all('tr')

After we create the variables, we setup lists to store the data of each column after processing

In [6]:
#list for each column
postal_code_list = []
borough_list = []
neighbourhood_list = []

In the next step we define a function to clean the data before we process it, the processed data is then iteratively appended to the lists we createed in the above cells

In [7]:
#creating a function to extract the text from the List_op_rows
def text_extractor(ele):
    temp_str = ele.get_text()
    temp_lst = temp_str.replace('\n','')
    return temp_lst
#print(temp_lst)
#printing elements from the table
for row in rows:
    line = row.find_all('td')
    counter_i = 0
    for ele in line:
        if counter_i == 0:
            postal_code_list.append(text_extractor(ele))
        elif counter_i == 1:
            borough_list.append(text_extractor(ele))
        elif counter_i == 2:
            neighbourhood_list.append(text_extractor(ele))
        else:
            pass
        counter_i = counter_i + 1
#printing the results of the processing
print(postal_code_list[0:5])
print(borough_list[0:5])
print(neighbourhood_list[0:5])


['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Regent Park, Harbourfront']


In the above output section, we see the first 5 rows of the lists we set up for each column in the table. Now we will dump this data into a Pandas DataFrame

In [8]:
dict_df = {'Postal Code': postal_code_list,'Borough':borough_list,'Neighbourhoods':neighbourhood_list}
toronto_df = pd.DataFrame.from_dict(dict_df)

We print the result of the data frame

In [9]:
print(toronto_df.head())
print('---------------------------------------------')
print(toronto_df.info())
print('----------------------------------------------')

  Postal Code           Borough             Neighbourhoods
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
---------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Postal Code     180 non-null    object
 1   Borough         180 non-null    object
 2   Neighbourhoods  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB
None
----------------------------------------------


Copying our Data Frame as a checkpoint in case we need to retrace our steps

In [10]:
checkpoint = toronto_df.copy()

We will now clean the data frame as per the requirements laid down in the assignement:
1. To ignore all the rows where boroughs are Not Assigned
2. If the neighbourhood is not assigned but the borough exists, then we will consider the borough as the neighbourhood

In [11]:
toronto_df.drop(toronto_df.loc[toronto_df['Borough']=='Not assigned'].index,axis = 0,inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhoods
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
toronto_df.reset_index(drop = True, inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
checkpoint_1 = toronto_df.copy()

In [14]:
toronto_df.loc[toronto_df['Neighbourhoods']=='Not Assigned']

,Postal Code,Borough,Neighbourhoods


There are no Neighbourhoods where the data is 'Not assigned', therefore as requested we obtain the shape of the data frame.

In [15]:
toronto_df.shape

(103, 3)

## Obtaining the Latitude and Longitude for each Neighbourhood.

In [17]:
!pip install geocoder

In [16]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code = 'M4A'
# loop until you get the coordinates
#while(lat_lng_coords is None):
g = geocoder.google("{},Toronto".format(postal_code))
lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

TypeError: 'NoneType' object is not subscriptable

In [44]:
print(latitude,longitude)

49.4842897 8.46700036809857


Due to failure from repeated attempts at calling the Geocoder API, we will npw use the file provided in the link http://cocl.us/Geospatial_data

In [17]:
latlng = pd.read_csv('http://cocl.us/Geospatial_data')
latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We copy the latlng df to a file_df incase we need to retrace our steps

In [18]:
file_df = latlng.copy()

Now we merge the Toronto Data Frame and the Latitde and Longititude DF

In [19]:
Toronto = toronto_df.merge(file_df, on = 'Postal Code')
Toronto.head()

,Postal Code,Borough,Neighbourhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
